<a href="https://colab.research.google.com/github/gabisouzaes/AnalisadorReunioesDevComGemini/blob/main/AnalisadorReunioesDevComGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
!pip install -q google-generativeai
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [100]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [101]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [102]:
# Importa as bibliotecas

# --- Importações Padrão do Python ---
import os  # Para interagir com o sistema operacional (ex: variáveis de ambiente)
import json  # Para trabalhar com dados no formato JSON

# --- Importações para Interação com APIs do Google (incluindo Google Docs) ---
# Biblioteca principal para construir serviços para APIs do Google
from googleapiclient.discovery import build

# Bibliotecas para autenticação no Google Cloud e Colab
from google.colab import auth, userdata  # 'auth' para autenticar o usuário no Colab, 'userdata' para acessar Secrets
import google.auth # Para obter as credenciais padrão após a autenticação

# --- Importações para o Google Generative AI (Gemini) ---
import google.generativeai as genai_sdk # SDK Padrão do Google Gemini, nomeado como 'genai_sdk'

# --- Importações para Exibição no Colab (Opcional, mas útil) ---
from IPython.display import Markdown, display # Para formatar e exibir saídas de forma mais rica

# --- Importação para Datas (Opcional, mas útil para registrar quando as análises são feitas) ---
from datetime import datetime

print("Todas as bibliotecas e funções necessárias foram importadas.")
print("Lembre-se de configurar sua API Key do Gemini e autenticar o usuário para o Google Docs.")

Todas as bibliotecas e funções necessárias foram importadas.
Lembre-se de configurar sua API Key do Gemini e autenticar o usuário para o Google Docs.


In [103]:
TECH_LEAD_NAME = "Gabriella"
NOME_MODELO_GEMINI_STRING = "gemini-2.0-flash"

In [104]:
# Inicializa o serviço do Google Docs
try:
    auth.authenticate_user()
    print("Usuário autenticado para serviços Google.")
    creds, project = google.auth.default()
    # Inicializar o serviço do Google Docs
    docs_service = build('docs', 'v1', credentials=creds)
    print("Serviço do Google Docs pronto para uso.")
except Exception as e:
    print(f"Erro na autenticação do Google: {e}")

Usuário autenticado para serviços Google.
Serviço do Google Docs pronto para uso.


In [113]:
PROMPT_INSIGHTS = """Você é uma assistente de IA com especialidade em identificar tendências de desenvolvimento em séries de conversas 1:1.
As reuniões analisadas são entre Gabriella (líder técnica) e um de seus liderados.
Interprete o conjunto de anotações de reuniões fornecido abaixo como um todo, refletindo a jornada de desenvolvimento deste liderado específico com Gabriella.

Com base em todas as anotações fornecidas, elabore um resumo com suas percepções gerais sobre os pontos de desenvolvimento do liderado. Em seu resumo, procure destacar:

* As principais áreas de foco para desenvolvimento (técnicas e comportamentais) que se mantiveram ao longo do tempo.
* Evidências claras de progresso ou, alternativamente, áreas onde o desenvolvimento pareceu estagnado ou enfrentou dificuldades.
* Desafios ou obstáculos que foram mencionados ou observados de forma recorrente.
* De 1 a 3 recomendações principais ou próximos passos para o desenvolvimento futuro do liderado.

Apresente suas percepções de forma clara, direta e concisa.
"""

def gerar_insights_de_documentos_agrupados(
    lista_ids,
    servico_docs,
    nome_lider,
    template_prompt,
    modelo_gemini_obj
):
    # Lê e combina o conteúdo dos documentos.
    textos_docs = []
    for doc_id in lista_ids:
        texto = ler_transcricao_do_google_docs(doc_id, servico_docs)
    textos_combinados = "".join(textos_docs)

    # Formata o prompt final com os dados coletados.
    prompt_final = template_prompt.format(
        nome_da_lider=nome_lider,
        textos_combinados_das_reunioes=textos_combinados
    )
    # Envia o prompt para o modelo Gemini e obtém a resposta.
    resposta = modelo_gemini_obj.generate_content(prompt_final)
    return resposta.text

# FIM DO TRECHO DE DEFINIÇÃO DA FUNÇÃO E PROMPT

# --- EXECUÇÃO PRINCIPAL DA ANÁLISE ---
print("\n--- Execução Principal da Análise ---")

# Verifica se os serviços essenciais foram inicializados corretamente.
if docs_service and modelo_gemini_objeto:
    # Coleta os IDs dos documentos do usuário. Incluir apenas ID do googledocs e não o link completo
    lista_de_document_ids = obter_lista_de_ids_google_docs()

    if lista_de_document_ids: # Procede apenas se o usuário forneceu algum ID.
        print("\n   Iniciando a geração de insights agrupados com o Gemini...")

        # INÍCIO DO BLOCO DE CHAMADA
        insights_finais = gerar_insights_de_documentos_agrupados(
            lista_de_document_ids, # Passa a lista de IDs coletada
            docs_service,
            TECH_LEAD_NAME,      # "Gabriella"
            PROMPT_INSIGHTS,
            modelo_gemini_objeto
        )

        # Exibe o resultado diretamente.
        # Se 'insights_finais' contiver uma string de erro da função, ela será exibida.
        print("\n   --- RESULTADO DA ANÁLISE DA IA ---")
        display(Markdown(str(insights_finais))) # Converte para str para segurança antes de Markdown

print("\n--- Fim do Script ---")


--- Execução Principal da Análise ---

   --- Coleta de IDs de Documentos do Google Docs ---
   Insira o ID do Documento nº 1 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1UMd1lqlvB_VWfW442obW0yjD3tlMt_7U55sxntLA3ho
   ID '1UMd1lqlvB_VWfW442obW0yjD3tlMt_7U55sxntLA3ho' adicionado à lista.
   Insira o ID do Documento nº 2 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1_SH0o76SJD9_htSe8S5WegYR08sjzc89fPr0uhp1bjo
   ID '1_SH0o76SJD9_htSe8S5WegYR08sjzc89fPr0uhp1bjo' adicionado à lista.
   Insira o ID do Documento nº 3 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1HVlICG87FCSLyRxqrZM84Umx7YoqgiV-lV6EC2CQfLA
   ID '1HVlICG87FCSLyRxqrZM84Umx7YoqgiV-lV6EC2CQfLA' adicionado à lista.
   Insira o ID do Documento nº 4 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1i7lcXMHI4LT9DZuUQ5_sEOMrFeC8vCi6GTrWUdMFUP8
   ID '1i7lcXMHI4LT

## Análise da Jornada de Desenvolvimento do Liderado de Gabriella

Com base nas anotações de reuniões fornecidas, elaborei um resumo com minhas percepções sobre a jornada de desenvolvimento do liderado, destacando as principais áreas de foco, progressos, desafios e recomendações.

**Resumo Geral:**

A jornada de desenvolvimento do liderado, guiada por Gabriella, parece focar em duas áreas principais: **habilidades de comunicação e autonomia técnica**. Inicialmente, a comunicação, tanto em termos de clareza quanto de proatividade em compartilhar informações, é um ponto central.  A autonomia técnica, por sua vez, está ligada à capacidade de resolver problemas de forma independente e tomar decisões com confiança.

**Áreas de Foco Recorrentes:**

*   **Comunicação:** Melhorar a clareza na comunicação de problemas e soluções, ser mais proativo em manter todos informados sobre o progresso do trabalho, e apresentar ideias de forma mais estruturada.
*   **Autonomia Técnica:** Desenvolver maior confiança para tomar decisões técnicas independentes e buscar soluções para problemas sem depender excessivamente de orientação.

**Evidências de Progresso e Dificuldades:**

*   **Progresso:** Há sinais de progresso na área de comunicação, com menções a melhorias na clareza e na proatividade. O liderado parece estar mais consciente da importância de manter a equipe informada. Pequenos incrementos na tomada de decisões técnicas também podem ser notados.
*   **Dificuldades/Estagnação:** A autonomia técnica parece ser uma área mais desafiadora. Embora haja tentativas de delegar decisões, o liderado demonstra insegurança e busca validação frequente, indicando uma dificuldade em internalizar o processo de tomada de decisão.

**Desafios e Obstáculos Recorrentes:**

*   **Insegurança Técnica:** A falta de confiança em suas próprias habilidades técnicas é um obstáculo recorrente. Isso leva à busca constante por validação e à dificuldade em tomar decisões de forma autônoma.
*   **Comunicação Reativa:** Apesar da melhora na proatividade, a comunicação ainda parece ser mais reativa do que proativa, especialmente em situações de problemas ou incertezas.
* **Dificuldade em estruturar ideias:** A apresentação de ideias e soluções ainda carece de estrutura, o que dificulta a compreensão e o engajamento da equipe.

**Recomendações e Próximos Passos:**

1.  **Foco em pequenas vitórias:** Definir projetos menores e bem definidos que permitam ao liderado tomar decisões técnicas com menor impacto. Isso pode aumentar a confiança e fomentar a autonomia. Gabriella pode atuar como um *safety net*, dando suporte, mas incentivando a tomada de decisão.
2.  **Simulações e role-playing:** Praticar a apresentação de ideias e soluções em um ambiente seguro. Isso pode ajudar o liderado a estruturar seus pensamentos e a comunicar suas ideias de forma mais eficaz.
3.  **Feedback estruturado e frequente:** Gabriella deve continuar fornecendo feedback claro e específico, tanto sobre os pontos fortes quanto sobre as áreas que precisam de melhoria. O feedback deve ser sempre orientado para o desenvolvimento, com foco em ações concretas que o liderado pode tomar.

Em resumo, a jornada de desenvolvimento do liderado está progredindo, mas a autonomia técnica continua sendo um desafio significativo. Ao focar em pequenas vitórias, praticar a apresentação de ideias e fornecer feedback estruturado, Gabriella pode ajudar o liderado a superar seus obstáculos e alcançar seu potencial máximo.



--- Fim do Script ---
